In [ ]:
import pandas as pd

# Ejercicio 1 y 2

In [ ]:
consumo_diario_petroleo = [320,200,785,376,2405,893,1930,8010,1540]
numero_horas_maquinas = [20,10,50,22,120,40,93,380,75]
distancia_transporte = [630,398,1395,720,4230,1752,4000,15780,3080]
rendimiento_promedio = [2.4,4.0,1.8,2.6,0.6,0.1,0.05,0.01,0.04]

In [ ]:
df = pd.DataFrame({
    "Consumo diario de petróleo": consumo_diario_petroleo,
    "Número de horas maquinas": numero_horas_maquinas,
    "Distancia de transporte": distancia_transporte,
    "Rendimiento promedio de motores": rendimiento_promedio
})

***
Realice la estadística descriptiva de las variables que se presentan.  
Agregue gráficos y un breve reporte de lo que observa.
***

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from pypalettes import get_hex

In [ ]:
for column in df.columns:
    fig = plt.figure(figsize =(4, 3))
    plt.boxplot(df[column])
    plt.title(column)
    plt.show()

In [ ]:
correlation_matrix = df.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Mapa de Calor de la Correlación')
plt.show()

In [ ]:
for column in df.columns:
    fig = plt.figure(figsize =(4, 3))
    plt.hist(df[column])
    plt.title(column)
plt.show()

Se puede apreciar que las variables tienen una correlación bastante elevada entre ellos, de hecho las parejas: **Consumo diario de petróleo** con **Número de horas maquinas**, **Consumo diario de petróleo** con **Distancia de transporte** y **Distancia de transporte** con **Número de horas maquinas** tienen una relación lineal perfecta, por otro lado también se visualiza la presencia de datos muy atípicos en las variables, por lo que en los diagramas de caja y brazos tenemos un valores superiores al brazo derecho en la mayoria de los casos, además ninguna de las variables se distribuye de forma normal

***
Ajuste la función de regresión múltiple que permite predecir el consumo de petróleo e indique si utilizarla para dicho fin,  
es adecuado o no, justifique su respuesta.
***

In [ ]:
import statsmodels.api as sm

In [ ]:
X = ["Número de horas maquinas", "Distancia de transporte", "Rendimiento promedio de motores"]
Y = ["Consumo diario de petróleo"]

In [ ]:
x = sm.add_constant(df[X])
model = sm.OLS(df[Y], x).fit()

In [ ]:
print(model.summary())

Para este resultado se obtiene que el predictor **Rendimiento promedio de motores** no es estadísticamente diferente de 0, además la $R^2=1$ y la ajustada a $0.99$, el modelo tiene claras señales de sobreajuste

***
¿Cuál de las tres variables independientes aporta más a la explicación 
del consumo de petróleo? Justifique su respuesta.
***

El **Número de horas maquinas** ya que tiene el coeficiente $B_i$ más alto, indicando su importancia para el modelo además de tener un $p_valor < 0.05$ estadísticamente significativo para el modelo

***
Explique el significado de los coeficientes de las variables: números de horas-  
máquinas y rendimiento de los motores, en el contexto del problema.
***

El número de horas-maquinas tienen un coefieciente de 11.2995 por lo que cada unidad hora-máquina esto contribuye en 11.2995 unidades en el consumo diario de gasolina, mientras que con rendimiento de los motores al tener un coeficiente de -4.7077 se puede inferir, que a medida que el rendimiento es mayor, menos gasolina se requerirá ára el proceso

***
Si sabe que en un día determinado las máquinas trabajarán durante 40 horas y
la distancia total a recorrer por los camiones será de 1000 km., ¿cuál debe ser
el rendimiento promedio de los motores de modo que el consumo de petróleo
no supere los 800 litros con un 95% de seguridad, sabiendo que el error
poblacional es sólo un 5% mayor que el error muestral de estimación?
***

In [ ]:
df = pd.DataFrame({
    "Consumo diario de petróleo": consumo_diario_petroleo,
    "Número de horas maquinas": numero_horas_maquinas,
    "Distancia de transporte": distancia_transporte,
    "Rendimiento promedio de motores": rendimiento_promedio
})

In [ ]:
coef = model.params

# Valores conocidos
horas_maquina = 40
distancia = 1000
consumo_maximo = 800

# Despejar el rendimiento
rendimiento = (consumo_maximo - coef['const'] - coef["Número de horas maquinas"] * horas_maquina - coef["Distancia de transporte"] * distancia) / coef["Rendimiento promedio de motores"]

In [ ]:
print(f'Rendimiento necesario para que el consumo no supere los 800 litros: {rendimiento:.2f} horas por litro')

***
Calcule los coeficientes estándares de las tres variables independientes e
indique si a través de los resultados obtenidos puede llegare a la misma
conclusión de a.
***

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[X])

In [ ]:
X_scaled = sm.add_constant(X_scaled)

In [ ]:
model_scaled = sm.OLS(df[Y], X_scaled).fit()

# Obtener el resumen del modelo
print(model_scaled.summary())

El coeficiente independiente pasa a ser singificativo, sin embargo, el **Rendimiento promedio de motores** se mantiene igual, además se sigue con un $R^2$ demasiado alto, llego a unas conclusiones muy similares que con el inciso a)

# Ejercicio 3
Los siguientes datos representa el costo de colegiaturas ( en miles de pesos) de  
una muestra de universidades privadas en diversas regiones de México. A nivel de  
significancia .05, ¿ Puede concluirse que existe alguna diferencia en el costo  
promedio de las colegiaturas?

In [ ]:
df2 = pd.DataFrame({
    "Region": ["Noreste"]*5 + ["Sureste"]*4 + ["Oeste"] * 5,
    "Costo": [10,11,12,10,12] + [8,9,10,8] + [7,8,6,7,6]
})

In [ ]:
import pingouin as pg

In [ ]:
# Test ANOVA de un factor
pg.welch_anova(
    data     = df2,
    dv       = 'Costo',
    between  = "Region"
)

Como `p-unc` es menor a 0.05 se concluye que existe una diferencia significativa entre el precio de las colegiaturas